In [1]:
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer

In [2]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai

In [3]:
load_dotenv()

True

In [4]:
import os

In [5]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [6]:
api_key = os.getenv("GOOGLE_API_KEY")

In [11]:
import google.generativeai as genai

genai.configure(api_key=api_key)

In [12]:
model = genai.GenerativeModel("gemini-2.5-flash")
response = model.generate_content("Xin chào, bạn có hoạt động không?")
print(response.text)

Chào bạn! Tôi đang hoạt động và sẵn sàng hỗ trợ bạn đây. Bạn có điều gì muốn hỏi hay cần giúp đỡ không?


In [29]:
df = pd.read_csv("./tmdb_5000_movies.csv", encoding="utf-8")

In [30]:
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [13]:
from transformers import pipeline

In [14]:
from pinecone import Pinecone

In [15]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("movies")

In [16]:
TRANSLATION_MODEL = "Helsinki-NLP/opus-mt-vi-en"

translator = pipeline("translation", model=TRANSLATION_MODEL)

D:\ai-env\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [17]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [18]:
def find_similar_movies_final(vietnamese_query, top_k=5):
    """
    Hàm tìm kiếm phiên bản cuối cùng:
    1. Dịch câu truy vấn Tiếng Việt sang Tiếng Anh.
    2. Dùng câu Tiếng Anh để tạo vector và tìm kiếm.
    """
    print("\n" + "="*50)
    print(f"🎬 Bắt đầu tìm kiếm cho query (VI): '{vietnamese_query}'")
    
    # BƯỚC 1: DỊCH SANG TIẾNG ANH
    translated_result = translator(vietnamese_query)
    english_query = translated_result[0]['translation_text']
    print(f"   -> Đã dịch sang (EN): '{english_query}'")
    
    # BƯỚC 2: TẠO VECTOR TỪ CÂU TIẾNG ANH
    query_embedding = model.encode(english_query).tolist() # Đảm bảo bạn dùng đúng tên biến 'embedding_model'
    
    # BƯỚC 3: TRUY VẤN PINECONE
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # BƯỚC 4: IN KẾT QUẢ (ĐÃ CẬP NHẬT)
    print("\n✨ Kết quả đề xuất: ✨")
    if not results['matches']:
        print("   Không tìm thấy phim phù hợp.")
        return

    for match in results['matches']:
        # Lấy dữ liệu từ metadata
        title = match['metadata'].get('title', 'N/A')
        score = match['score']
        
        # --- DÒNG MỚI ĐƯỢC THÊM VÀO ---
        genres = match['metadata'].get('genres', 'Không rõ') # Lấy genres, nếu không có thì ghi 'Không rõ'
        
        # --- ĐÃ CẬP NHẬT LỆNH PRINT ---
        print(f"  - Tiêu đề: {title} (Score: {score:.2f})")
        print(f"    Thể loại: {genres}") # In ra thể loại
        
        # Bạn có thể bỏ comment dòng dưới nếu muốn xem cả tóm tắt
        # overview = match['metadata'].get('overview', 'No overview available.')
        # print(f"    Tóm tắt: {overview[:100]}...")
        print("") # Thêm một dòng trống cho dễ đọc

    print("="*50)

In [19]:
find_similar_movies_final("Phim về chiếc thuyền bị đắm")


🎬 Bắt đầu tìm kiếm cho query (VI): 'Phim về chiếc thuyền bị đắm'
   -> Đã dịch sang (EN): 'The movie about the boat sinking.'

✨ Kết quả đề xuất: ✨
  - Tiêu đề: The Reef (Score: 0.59)
    Thể loại: Drama, Horror, Thriller

  - Tiêu đề: In the Heart of the Sea (Score: 0.59)
    Thể loại: Thriller, Drama, Adventure, Action, History

  - Tiêu đề: Titanic (Score: 0.59)
    Thể loại: Drama, Romance, Thriller

  - Tiêu đề: Abandoned (Score: 0.58)
    Thể loại: Drama, Thriller

  - Tiêu đề: All Is Lost (Score: 0.55)
    Thể loại: Action, Adventure, Drama



# RAG

In [22]:
def find_similar_movies_for_rag(vietnamese_query, top_k=3):
    """
    Hàm này được sửa lại để TRẢ VỀ (return) dữ liệu cho RAG, 
    thay vì in ra.
    """
    print(f"Bắt đầu Retrieval cho: '{vietnamese_query}'")
    
    # BƯỚC 1: DỊCH
    translated_result = translator(vietnamese_query)
    english_query = translated_result[0]['translation_text']
    
    # BƯỚC 2: TẠO VECTOR
    query_embedding = model.encode(english_query).tolist()
    
    # BƯỚC 3: TRUY VẤN PINECONE
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    # BƯỚC 4: TẠO CONTEXT (thay vì in)
    movie_context = []
    if not results['matches']:
        return None # Không tìm thấy gì

    for match in results['matches']:
        title = match['metadata'].get('title', 'N/A')
        genres = match['metadata'].get('genres', 'Không rõ')
        movie_context.append(f" - Tiêu đề: {title} (Thể loại: {genres})")
    
    # Trả về một chuỗi context sạch sẽ
    return "\n".join(movie_context)

In [27]:
api_key = os.getenv("GOOGLE_API_KEY") 
genai.configure(api_key=api_key)

# Cấu hình mô hình
generation_config = {
  "temperature": 0.7,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}
model_g = genai.GenerativeModel(model_name="gemini-2.5-flash", # Hoặc gemini-pro
                              generation_config=generation_config)

def generate_rag_response(vietnamese_query, movie_context):
    """
    Hàm này chính là chữ "G" - Generation.
    """
    if movie_context is None:
        return "Xin lỗi, tôi không tìm thấy bộ phim nào phù hợp với yêu cầu của bạn."

    # Đây là trái tim của RAG: Prompt "nhồi" thêm context
    prompt = f"""
    Bạn là một trợ lý AI tư vấn phim ảnh rất thân thiện và am hiểu.
    
    Người dùng vừa hỏi: "{vietnamese_query}"
    
    Tôi đã tìm trong cơ sở dữ liệu và thấy một số phim liên quan sau đây:
    ---
    {movie_context}
    ---
    
    Dựa vào thông tin trên, hãy viết một câu trả lời tư vấn ngắn gọn,
    thân thiện và tự nhiên cho người dùng (bằng Tiếng Việt).
    """
    
    print("\nĐang thực hiện Generation")
    response = model_g.generate_content(prompt)
    return response.text

In [29]:
query = "Cho tôi vài phim về du hành thời gian đi"

retrieved_context = find_similar_movies_for_rag(query)

final_answer = generate_rag_response(query, retrieved_context)

# 3. In ra câu trả lời cuối cùng
print("\n: Câu trả lời RAG:")
print(final_answer)

Bắt đầu Retrieval cho: 'Cho tôi vài phim về du hành thời gian đi'

Đang thực hiện Generation

: Câu trả lời RAG:
Chào bạn! Chủ đề du hành thời gian nghe hấp dẫn quá! 🚀

Mình đã tìm được vài phim rất hay cho bạn đây, với nhiều thể loại khác nhau để bạn lựa chọn nhé:

*   Nếu bạn muốn một chút hài hước và giả tưởng nhẹ nhàng, có thể thử **The Visitors II: The Corridors of Time**.
*   Còn nếu bạn thích những bộ phim khoa học viễn tưởng cân não, đầy kịch tính và giật gân, thì **Timecrimes** sẽ là một lựa chọn đáng xem đó!
*   Và tất nhiên, làm sao có thể bỏ qua một phần của series kinh điển về du hành thời gian như **Back to the Future Part II** chứ? Phim này có đủ phiêu lưu, hài hước và khoa học viễn tưởng luôn!

Hy vọng bạn sẽ tìm được bộ phim ưng ý để "xuyên không" cuối tuần này nhé! Chúc bạn xem phim vui vẻ! ✨


# Main

## Vector Embeddings

In [18]:
import os
from transformers import pipeline
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import google.generativeai as genai

# --- 1. CÀI ĐẶT VÀ TẢI MODEL (Giữ nguyên code của bạn) ---

# Lấy API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Cấu hình Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("movies") # Đảm bảo tên index chính xác

# Cấu hình Google GenAI (Cần thiết cho LangChain)
genai.configure(api_key=GOOGLE_API_KEY)

# Tải các model (HuggingFace & SentenceTransformer)
print("Đang tải model...")
TRANSLATION_MODEL = "Helsinki-NLP/opus-mt-vi-en"
translator = pipeline("translation", model=TRANSLATION_MODEL)
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
print("Tải model hoàn tất.")

Đang tải model...


Device set to use cpu


Tải model hoàn tất.


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [48]:

def translate_query(vietnamese_query):
    print(f"[LCEL] 1. Đang dịch: '{vietnamese_query}'")
    translated_result = translator(vietnamese_query)
    english_query = translated_result[0]['translation_text']
    print(f"  -> Đã dịch: '{english_query}'")
    return english_query

translate_step = RunnableLambda(translate_query)


def get_embedding(english_query):
    print(f"[LCEL] 2. Đang tạo vector cho: '{english_query}'")
    return embedding_model.encode(english_query).tolist()

embed_step = RunnableLambda(get_embedding)

def query_pinecone(query_embedding):
    print("[LCEL] 3. Đang truy vấn Pinecone...")
    results = index.query(
        vector=query_embedding,
        top_k=10, # Lấy 10 kết quả như bạn muốn
        include_metadata=True
    )
    print(f"  -> Tìm thấy {len(results['matches'])} kết quả.")
    return results['matches'] # Chỉ trả về danh sách matches

query_pinecone_step = RunnableLambda(query_pinecone)

def format_context_from_pinecone(matches):
    """
    Hàm 'chuẩn hóa' này được cập nhật cho Pinecone
    để bao gồm scores và thể loại.
    """
    print("[LCEL] 2. Đang chuẩn hóa Context từ Pinecone...")
    
    if not matches:
        return "Xin lỗi, không tìm thấy phim phù hợp trên Pinecone."

    context = "Dưới đây là các bộ phim gợi ý tìm thấy từ Pinecone:\n\n"
    
    # Lặp qua từng 'match' do Pinecone trả về
    for i, match in enumerate(matches):
        
        # --- LẤY THÔNG TIN ĐÃ CẬP NHẬT ---
        
        # 1. Lấy điểm Score (nằm ở cấp ngoài cùng)
        score = match['score'] 
        
        # 2. Lấy metadata
        metadata = match.get('metadata', {}) # Lấy metadata an toàn
        
        # 3. Lấy thông tin từ metadata
        title = metadata.get('title', 'N/A')
        # (Quan trọng: Giả định metadata của bạn có trường 'genres')
        genres = metadata.get('genres', 'Không rõ') 
        overview = metadata.get('overview', 'Không có tóm tắt.')
        
        # --- TẠO CONTEXT STRING ---
        context += f"--- Phim {i+1} ---\n"
        context += f"Tiêu đề: {title}\n"
        context += f"Điểm tương đồng (Score): {score:.2f}\n" # Đã thêm
        context += f"Thể loại: {genres}\n" # Đã thêm
        context += f"Tóm tắt: {overview}\n\n"
    
    return context

# Tạo lại RunnableLambda với hàm mới
augmentation_step = RunnableLambda(format_context_from_pinecone)

In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

In [53]:
prompt_template = """
Bạn là một trợ lý tư vấn phim ảnh chuyên nghiệp. Người dùng đã cung cấp tên một bộ phim, và đây là danh sách các phim tương tự nhất được tìm thấy.

**BỐI CẢNH (Danh sách phim gợi ý đã được chuẩn hóa):**
{context}

**PHIM GỐC MÀ NGƯỜI DÙNG HỎI:**
"{question}"

**NHIỆM VỤ:**
Hãy viết một câu trả lời thân thiện bằng Tiếng Việt với cấu trúc sau:

1. CHÀO HỎI: Chào người dùng và xác nhận bộ phim họ đang quan tâm
2. GIỚI THIỆU PHIM GỢI Ý: Giới thiệu 2-3 bộ phim nổi bật nhất từ BỐI CẢNH
3. THÔNG TIN CHI TIẾT: Với mỗi phim, cung cấp đầy đủ:
   - **Điểm tương đồng** (Score)
   - **Thể loại** chính
   - **Tóm tắt nội dung** ngắn gọn
   - **Lý do tương tự** dựa trên thể loại và tóm tắt

**QUY TẮC QUAN TRỌNG:**
- CHỈ sử dụng thông tin từ BỐI CẢNH được cung cấp
- LUÔN bao gồm điểm số tương đồng, thể loại và tóm tắt cho mỗi phim
- Giải thích ngắn gọn tại sao phim đó tương tự với phim gốc
- Giữ giọng văn thân thiện, nhiệt tình
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

parser = StrOutputParser()

retrieval_and_augmentation_chain = (
    translate_step 
    | embed_step 
    | query_pinecone_step 
    | augmentation_step  
)

# Chain RAG chính
# (Prompt cũng nên được cập nhật để LLM biết dùng 'Thể loại' và 'Score')
rag_chain = (
    {"context": retrieval_and_augmentation_chain, "question": RunnablePassthrough()}
    | prompt # (Prompt này nên được cập nhật)
    | llm
    | parser
)

In [55]:
query = "Chiếc thuyền bị đắm"
final_recommendation = rag_chain.invoke(query)
print(final_recommendation)


[LCEL] 1. Đang dịch: 'Chiếc thuyền bị đắm'
  -> Đã dịch: 'The boat sank.'
[LCEL] 2. Đang tạo vector cho: 'The boat sank.'
[LCEL] 3. Đang truy vấn Pinecone...
  -> Tìm thấy 10 kết quả.
[LCEL] 2. Đang chuẩn hóa Context từ Pinecone...
Chào bạn! Tôi rất vui được giúp bạn tìm kiếm những bộ phim tương tự với "Chiếc thuyền bị đắm". Dựa trên thông tin tôi có, đây là một vài gợi ý tuyệt vời mà bạn có thể thích:

---

**1. In the Heart of the Sea**
*   **Điểm tương đồng:** 0.50
*   **Thể loại chính:** Thriller, Drama, Adventure
*   **Tóm tắt nội dung:** Vào mùa đông năm 1820, con tàu săn cá voi Essex của vùng New England bị tấn công bởi một con cá voi khổng lồ với ý chí trả thù gần như của con người. Thảm họa hàng hải có thật này đã truyền cảm hứng cho tác phẩm Moby Dick của Herman Melville. Bộ phim hé lộ hậu quả kinh hoàng của cuộc chạm trán, khi thủy thủ đoàn sống sót bị đẩy đến giới hạn và buộc phải làm những điều không tưởng để sống sót.
*   **Lý do tương tự:** Bộ phim này có điểm tương đồng

# Tf_IDF

In [25]:
import pandas as pd
import json
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
import dill

with open('movie_recommender_2.pkl', 'rb') as file:
    movie_recommender = dill.load(file)


In [27]:
print(movie_recommender.recommend('The Dark Knight'))


119                         Batman Begins
428                        Batman Returns
3                   The Dark Knight Rises
9      Batman v Superman: Dawn of Justice
210                        Batman & Robin
Name: title, dtype: object


In [32]:
recommendations = movie_recommender.recommend("Batman")

In [38]:
full_info = df.loc[119]

print(full_info)

budget                                                          150000000
genres                  [{"id": 28, "name": "Action"}, {"id": 80, "nam...
homepage                http://www2.warnerbros.com/batmanbegins/index....
id                                                                    272
keywords                [{"id": 486, "name": "himalaya"}, {"id": 779, ...
original_language                                                      en
original_title                                              Batman Begins
overview                Driven by tragedy, billionaire Bruce Wayne ded...
popularity                                                     115.040024
production_companies    [{"name": "DC Comics", "id": 429}, {"name": "L...
production_countries    [{"iso_3166_1": "GB", "name": "United Kingdom"...
release_date                                                   2005-06-10
revenue                                                         374218673
runtime                               

In [39]:
import pandas as pd
import json
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class MovieRecommender:
    def __init__(self, data_path):
        # Đọc dữ liệu
        self.df = pd.read_csv(data_path)
        
        # Xử lý genres và keywords thành chuỗi
        self.df['string'] = self.df.apply(self.genres_and_keywords_to_string, axis=1)
        
        # Tạo TF-IDF vectorizer và fit-transform dữ liệu
        self.tfidf = TfidfVectorizer(max_features=2000)
        self.X = self.tfidf.fit_transform(self.df['string'])
        
        # Tạo Series để dễ dàng truy xuất movie index
        self.movie_idx = pd.Series(self.df.index, index=self.df['title'])
    
    @staticmethod
    def genres_and_keywords_to_string(row):
        try:
            genres = json.loads(row['genres'])
            genres = ' '.join(''.join(j['name'].split()) for j in genres)
        except:
            genres = ""

        try:
            keywords = json.loads(row['keywords'])
            keywords = ' '.join(''.join(j['name'].split()) for j in keywords)
        except:
            keywords = ""
            
        return "%s %s" % (genres, keywords)
    
    def recommend(self, title):
        if title not in self.movie_idx:
            print(f"Không có bộ phim '{title}' trong danh sách.")
            # Trả về một DataFrame trống nếu không tìm thấy
            return pd.DataFrame() 
        else:
            idx = self.movie_idx[title]
            if type(idx) == pd.Series: # Kiểm tra xem có nhiều phim trùng tên không
                idx = idx.iloc[0]
            
            query = self.X[idx]
            scores = cosine_similarity(query, self.X)
            scores = scores.flatten()
            
            # Lấy 5 index hàng đầu (từ 1 đến 6)
            recommended_idx = (-scores).argsort()[1:6]
            
            # --- THAY ĐỔI QUAN TRỌNG BẮT ĐẦU TỪ ĐÂY ---
            
            # 1. Lấy DataFrame con của các phim được gợi ý
            recommended_movies_df = self.df.iloc[recommended_idx].copy()
            
            # 2. Lấy điểm số tương đồng của 5 phim đó
            recommended_scores = scores[recommended_idx]
            
            # 3. Thêm điểm số vào DataFrame để RAG có thể sử dụng
            recommended_movies_df['similarity_score'] = recommended_scores
            
            # 4. Trả về TOÀN BỘ DataFrame thay vì chỉ tên phim
            return recommended_movies_df

# --- Chạy lại để lưu file pickle MỚI ---
print("Đang tạo và lưu recommender đã cập nhật...")
movie_recommender = MovieRecommender('tmdb_5000_movies.csv')
with open('movie_recommender_2.pkl', 'wb') as file:
    pickle.dump(movie_recommender, file)
print("Đã lưu 'movie_recommender_2.pkl' (phiên bản cập nhật).")

Đang tạo và lưu recommender đã cập nhật...
Đã lưu 'movie_recommender_2.pkl' (phiên bản cập nhật).


In [46]:
import os
import pickle
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# --- 1. CÀI ĐẶT API VÀ LLM ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
parser = StrOutputParser()

# --- 2. TẢI RECOMMENDER ĐÃ CẬP NHẬT ---
print("Đang tải 'movie_recommender_2.pkl'...")
with open('movie_recommender_2.pkl', 'rb') as file:
    # 'movie_recommender' này là đối tượng đã được huấn luyện
    movie_recommender = pickle.load(file) 


# --- 3. CÁC HÀM HELPER "CHUẨN HÓA" (ĐÃ ĐƠN GIẢN HÓA) ---

def parse_genres(json_string, key='name'):
    """Helper để đọc JSON từ cột 'genres'"""
    try:
        items = json.loads(json_string)
        if not items:
            return "Không rõ"
        return ", ".join([item[key] for item in items])
    except (json.JSONDecodeError, TypeError):
        return "Không rõ"

# (Đã XÓA hàm parse_cast và parse_director)

# --- 4. ĐỊNH NGHĨA CÁC MẮT XÍCH (RUNNABLES) ---

# Mắt xích 1: Retrieval (R)
def retrieve_similar_movies(title: str):
    print(f"[LCEL] 1. Đang truy xuất (TF-IDF) cho: '{title}'")
    return movie_recommender.recommend(title) 

retrieval_step = RunnableLambda(retrieve_similar_movies)


# Mắt xích 2: Augmentation (A) - "Chuẩn hóa" (ĐÃ CẬP NHẬT)
def format_context_from_df(recommender_output_df: pd.DataFrame):
    """
    Hàm 'chuẩn hóa' này ĐÃ ĐƯỢC CẬP NHẬT
    để chỉ bao gồm scores, thể loại và tóm tắt.
    """
    print("[LCEL] 2. Đang chuẩn hóa DataFrame thành Context (Đơn giản)...")
    
    if recommender_output_df is None or recommender_output_df.empty:
        return "Xin lỗi, không tìm thấy phim gốc để đưa ra gợi ý."

    context = "Dưới đây là 5 bộ phim gợi ý dựa trên sự tương đồng về nội dung (thể loại, từ khóa):\n\n"
    
    for i, row in enumerate(recommender_output_df.iterrows()):
        movie = row[1] 
        
        # --- GỌI CÁC HÀM HELPER ---
        score = movie['similarity_score']
        genres = parse_genres(movie['genres'])
        
        # --- TẠO CONTEXT STRING (ĐÃ ĐƠN GIẢN HÓA) ---
        context += f"--- Phim {i+1} ---\n"
        context += f"Tiêu đề: {movie['title']}\n"
        context += f"Điểm tương đồng (Score): {score:.2f}\n"
        context += f"Thể loại: {genres}\n"
        context += f"Tóm tắt: {movie['overview']}\n\n"
    
    return context

augmentation_step = RunnableLambda(format_context_from_df)


# Mắt xích 3: Generation (G) - Prompt (ĐÃ CẬP NHẬT)
prompt_template = """
Bạn là một trợ lý tư vấn phim ảnh chuyên nghiệp. Người dùng đã cung cấp tên 
một bộ phim, và đây là danh sách các phim tương tự nhất được tìm thấy.

**BỐI CẢNH (Danh sách phim gợi ý đã được chuẩn hóa):**
{context}

**PHIM GỐC MÀ NGƯỜI DÙNG HỎI:**
"{question}"

**NHIỆM VỤ:**
Hãy viết một câu trả lời thân thiện bằng Tiếng Việt.
1. Chào người dùng và xác nhận bộ phim họ hỏi.
2. Dựa vào BỐI CẢNH, hãy giới thiệu 2-3 bộ phim nổi bật nhất.
3. Giải thích ngắn gọn tại sao chúng tương tự (dựa trên **thể loại** và **tóm tắt**).
"""
prompt = ChatPromptTemplate.from_template(prompt_template)


# --- 5. XÂY DỰNG VÀ CHẠY CHUỖI RAG ---

print("\n" + "="*50)
print("🔗 XÂY DỰNG CHUỖI LCEL (Phiên bản đơn giản) 🔗")

# Chain con: Retrieval + Augmentation
retrieval_and_augmentation_chain = retrieval_step | augmentation_step

# Chain RAG chính
rag_chain = (
    {"context": retrieval_and_augmentation_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

print("Đã xây dựng chain thành công!")

Đang tải 'movie_recommender_2.pkl'...

🔗 XÂY DỰNG CHUỖI LCEL (Phiên bản đơn giản) 🔗
Đã xây dựng chain thành công!


In [47]:
query = "The Dark Knight"
final_recommendation = rag_chain.invoke(query)

print(final_recommendation)

[LCEL] 1. Đang truy xuất (TF-IDF) cho: 'The Dark Knight'
[LCEL] 2. Đang chuẩn hóa DataFrame thành Context (Đơn giản)...
Chào bạn! Rất vui được hỗ trợ bạn tìm phim. Bạn đang tìm các bộ phim tương tự như **"The Dark Knight"** phải không? Đây là một lựa chọn tuyệt vời và tôi có một vài gợi ý đáng giá dành cho bạn đây!

Dựa trên sự tương đồng về nội dung, thể loại và từ khóa, tôi đặc biệt gợi ý hai bộ phim sau:

1.  **Batman Begins** (Điểm tương đồng: 0.50)
    *   **Tại sao tương tự:** Đây là phần mở đầu của bộ ba phim Hiệp sĩ bóng đêm của đạo diễn Christopher Nolan, cũng là bộ phim đặt nền móng cho phong cách thực tế và u tối mà "The Dark Knight" đã phát triển.
    *   **Về thể loại:** Cả hai phim đều thuộc thể loại **Hành động, Tội phạm, Chính kịch**, tập trung vào một thế giới siêu anh hùng có chiều sâu tâm lý và các vấn đề xã hội phức tạp.
    *   **Về nội dung:** "Batman Begins" khám phá hành trình Bruce Wayne trở thành Batman, nguồn gốc của nỗi sợ hãi và cuộc chiến đầu tiên của anh 